# Generate top of the book using dedicated service - C#

### Overview
Retrieving best bid/ask over a look back period is a valuable indicator to highlight market activity. It can be correlated with a set of technical indicators to better analyze market movements and build a decision making parameters.

This sample is designed to request top of the book prices and sizes and illustrate market activity over a look back period.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** Bid/Ask information over the time.

### Inputs/outputs
Trades extraction sample requires instrument's identifier, date time intervals as per inputs and returns the best bid/ask prices and sizes.

### Services used
This sample uses *gRPC requests* in order to retrieve top of the book tick data from the hosted service. The queried endpoint in this script are:
* *TickBookService*: to directly retrieve best Bid/Ask tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run Top of the book sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress(System.Environment.GetEnvironmentVariable("GRPC_APIS"));

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
var start = new DateTime(2021, 03, 05);
var end = new DateTime(2021, 03, 05);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
 StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
 EndTime = new TimeOfDay { Hours = 14, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create requests for *top of the book* data and instantiate the service:

In [ ]:
// Generate the tick book request
var request = new TickBookRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick book service
var service = new TickBookService.TickBookServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve top of the book data

The following code snippets create requests to retrieve *bid* and *ask* prices and volumes:

In [ ]:
// Get the limits
var limits = new List<BookLimit>();
var call = service.TickBook(request, headers);
await foreach (var book in call.ResponseStream.ReadAllAsync())
{
    if (book.Limit != null) 
    {
        limits.Add(book.Limit);
    } 
}
// Display top of the book
//display(limits);

#### 4.2 Plot top of the book data
Display *best bid* and *best ask* prices and volumes over the look back period with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
// Plot simultaneously Bid and Ask prices
var topOfBookPricesChart = new List<Scattergl>
{
    new Scattergl
    {
        name = "AskPrice",
        mode = "lines",
        x = limits.Select(elt => elt.TimeStamp.ToDateTime()),
        y = limits.Select(elt => elt.Ask.Price)
    },
    new Scattergl
    {
        name = "BidPrice",
        mode = "lines",
        x = limits.Select(elt => elt.TimeStamp.ToDateTime()),
        y = limits.Select(elt => elt.Bid.Price)
    }
};
var pricechart = Chart.Plot(topOfBookPricesChart);
pricechart.WithTitle($"Best Bid/Ask prices for {ticker}");
pricechart.WithXTitle($"Time");
pricechart.WithYTitle($"Price");
pricechart.Width = 1500;
pricechart.Height = 800;
display(pricechart);

// Plot simultaneously Bid and Ask sizes
var topOfBookSizesChart = new List<Scattergl>
{
    new Scattergl
    {
        name = "AskSize",
        mode = "lines",
        x = limits.Select(elt => elt.TimeStamp.ToDateTime()),
        y = limits.Select(elt => -elt.Ask.Size)
    },
    new Scattergl
    {
        name = "BidSize",
        mode = "lines",
        x = limits.Select(elt => elt.TimeStamp.ToDateTime()),
        y = limits.Select(elt => elt.Bid.Size)
    }
};
var sizechart = Chart.Plot(topOfBookSizesChart);
sizechart.WithTitle($"Best Bid/Ask sizes for {ticker}");
sizechart.WithXTitle($"Time");
sizechart.WithYTitle($"Size");
sizechart.Width = 1500;
sizechart.Height = 800;
display(sizechart);